In [1]:
import glob
import pandas as pd
from collections import defaultdict
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import numpy as np
from tqdm import tqdm
import string

/Users/themistoklis/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/themistoklis/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#read animal_cases.csv as df
df = pd.read_csv('../Datasets/animal_cases.csv')

/var/folders/qf/yh559xpx4dv12z0mkyw5cvr80000gn/T/ipykernel_3964/1177899459.py:2: DtypeWarning: Columns (13,16,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Datasets/animal_cases.csv')


To do this, we first need to manually check the format of "closure_reason". Employees give a closure status and then enter text indicating findings or work done to close a given case.

In [5]:
#print the number of rows that have "dog" in the "closure_reason" column
print(len(df[df['closure_reason'].str.contains('dog', case=False)]))
#print closure_reason of rows that have dog in closure_reason
list(df[df['closure_reason'].str.contains('dog', case=False)]['closure_reason'])[:10]

4801


['Case Closed Case Resolved  Sent a crew to the location and found nothing looked at the time it was called in on the night shift and they must of picked up the dog.  Russell Quigg  ',
 'Case Closed Case Resolved gm2 picked up dead dog ',
 'Case Closed Case Resolved  Dog is on PVT. property not on sidewalk I will send a crew back to pick up when the constituent removes the dog and puts it on the sidewalk.  Russell Quigg  ',
 'Case Closed Case Resolved gm2 picked up dead dog ',
 'Case Closed Case Noted Animal control took dog to marlowe st. Constituent can pick-up tomorrow. ',
 'Case Closed Case Resolved Dead animal removed (dog) ',
 'Case Closed Case Resolved there was no dead dog tobe found ',
 'Case Closed Case Resolved Dead dog removed by BPD  GM2 picked it up at So. Boston police station ',
 'Case Closed Case Resolved dist 1 picked up dead dog ',
 'Case Closed Case Resolved dog and cat were picked up. ']

Here is our initial methodology for extracting distinct animals from the "closure_reason" (we treat animals as nouns). Note that this dictionary that we create is expected to contain keys irrelevant to animals.

In [6]:
def extract_distinct_animals(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Perform part-of-speech tagging to identify nouns (NN: singular noun, NNS: plural noun)
    pos_tags = pos_tag(words)

    # Extract nouns from the tagged words
    animals = [word for word, pos in pos_tags if pos in ['NN', 'NNS']]

    # Remove duplicates
    distinct_animals = list(set(animals))

    return distinct_animals
#use this function to extract distinct animals from the "closure_reason" column, put the result in a dict where keys are the animals and values are the number of occurrences
animal_counts = defaultdict(int)
for closure_reason in df['closure_reason']:
    animals = extract_distinct_animals(closure_reason)
    for animal in animals:
        animal_counts[animal] += 1


In our endeavor to streamline the identification of animal-related entities and minimize manual effort, we implemented a filtering mechanism. Specifically, we excluded keys (or nouns) appearing in fewer than three requests, as well as nouns with a length less than two and those containing special characters.

In [7]:
#remove all keys that have value of < 3
animal_counts = {key: value for key, value in animal_counts.items() if value > 3}
#remove all keys of less than 2 characters
animal_counts = {key: value for key, value in animal_counts.items() if len(key) > 2}
#remove all keys with dots, slashes, numbers and - in them
animal_counts = {key: value for key, value in animal_counts.items() if '.' not in key and '/' not in key and not any(char.isdigit() for char in key) and '-' not in key}

sorted_animal_counts = dict(sorted(animal_counts.items(), key=lambda item: item[1], reverse=True))
for animal, count in sorted_animal_counts.items():
    print(animal, count)

date 54291
EDT 11227
animal 7079
location 6428
EST 4555
dog 3531
rat 2799
shelter 2346
case 2242
set 2157
owner 2105
cat 1912
bird 1908
squirrel 1601
states 1471
property 1458
arrival 1355
caller 1170
information 1167
area 1132
contact 1083
request 1027
control 866
please 865
raccoon 838
constituent 827
Team 827
rabbit 805
error 796
dogs 786
call 723
officer 664
creation 650
nothing 647
rodent 625
complaint 616
street 600
hours 591
address 581
skunk 558
time 550
phone 496
days 437
responsibility 426
bat 425
picture 416
message 407
sidewalk 344
seagull 339
minutes 314
eform 313
notice 305
report 294
number 288
crew 281
service 262
possum 260
park 258
pigeon 242
scene 240
owners 240
wildlife 233
morning 231
reports 231
Case 228
goose 222
day 218
Tue 212
requests 210
Munroe 210
Thu 209
citation 207
inconvenience 206
pick 204
confusion 204
anything 197
bpw 182
roadway 174
door 167
inspection 160
animals 159
pickup 154
home 154
turkey 148
way 146
hawk 146
thank 144
file 142
dist 140
duplica

In [9]:
# Manual cleaning of animals. We store the non animals in a seperate file to further investigate
# for every key in sorted_animal_counts, ask the user to add key and value to a new dict or not. Ask until the user types either "y" or "n"
animal_mapping = {}
other = {}
for animal, number in sorted_animal_counts.items():
    print(f'{animal}: {number}')
    response = input(f'Add "{animal}" to animal_mapping? (y/n): ')
    while response.lower() not in ['y', 'n']:
        print('Invalid response. Please enter "y" or "n".')
        response = input(f'Add "{animal}" to animal_mapping? (y/n): ')
    if response.lower() == 'y':
        animal_mapping[animal] = number
    else:
        other[animal] = number

date: 54291
EDT: 11227
animal: 7079
location: 6428
EST: 4555
dog: 3531
rat: 2799
shelter: 2346
case: 2242
set: 2157
owner: 2105
cat: 1912
bird: 1908
squirrel: 1601
states: 1471
property: 1458
arrival: 1355
caller: 1170
information: 1167
area: 1132
contact: 1083
request: 1027
control: 866
please: 865
raccoon: 838
constituent: 827
Team: 827
rabbit: 805
error: 796
dogs: 786
call: 723
officer: 664
creation: 650
nothing: 647
rodent: 625
complaint: 616
street: 600
hours: 591
address: 581
skunk: 558
time: 550
phone: 496
days: 437
responsibility: 426
bat: 425
picture: 416
message: 407
sidewalk: 344
seagull: 339
minutes: 314
eform: 313
notice: 305
report: 294
number: 288
crew: 281
service: 262
possum: 260
park: 258
pigeon: 242
scene: 240
owners: 240
wildlife: 233
morning: 231
reports: 231
Case: 228
goose: 222
day: 218
Tue: 212
requests: 210
Munroe: 210
Thu: 209
citation: 207
inconvenience: 206
pick: 204
confusion: 204
anything: 197
bpw: 182
roadway: 174
door: 167
inspection: 160
animals: 159
pi

In [90]:
# write other to a csv file
other_df = pd.DataFrame(other.items(), columns=['animal', 'count'])
other_df.to_csv('Datesets/non_animals.csv', index=False)

In [30]:
# for two consecutive keys in animal_mapping_copy, ask the user to merge them or not to a new dict until the user types either "y" or "n". If they merge, go to the next key and repeat the process
merged_animal_mapping = {}
keys = list(animal_mapping.keys())
i = 0
while i < len(keys) - 1:
    key1 = keys[i]
    key2 = keys[i + 1]
    print(f'{key1}: {animal_mapping[key1]}')
    print(f'{key2}: {animal_mapping[key2]}')
    response = input(f'Merge "{key1}" and "{key2}"? (y/n): ')
    while response.lower() not in ['y', 'n']:
        print('Invalid response. Please enter "y" or "n".')
        response = input(f'Merge "{key1}" and "{key2}"? (y/n): ')
    if response.lower() == 'y':
        merged_animal_mapping[f'{key1}, {key2}'] = animal_mapping[key1] + animal_mapping[key2]
        i += 2
    else:
        merged_animal_mapping[key1] = animal_mapping[key1]
        i += 1

Dog: 21
Gillis: 57
Gillis: 57
Munroe: 210
Munroe: 210
Rabbit: 7
Rabbit: 7
Raccoon: 11
Raccoon: 11
Turkey: 5
Turkey: 5
bat: 425
bat: 425
bats: 13
beaver: 10
bird: 1908
bird: 1908
birds: 75
bull: 19
bunnies: 4
bunnies: 4
bunny: 78
cat: 1912
cats: 84
chicken: 14
chickens: 9
chipmunk: 9
coyote: 82
coyote: 82
coyotes: 9
crow: 19
deer: 45
deer: 45
dist: 140
dist: 140
dog: 3531
dog: 3531
dogs: 786
dove: 20
duck: 54
duck: 54
ducks: 17
dumpster: 10
falcon: 5
falcon: 5
ferret: 8
ferret: 8
fish: 4
fish: 4
fox: 13
fox: 13
geese: 8
geese: 8
goose: 222
goose: 222
groundhog: 13
groundhog: 13
gull: 95
gull: 95
gutter: 13
gutter: 13
gutters: 5
hawk: 146
hog: 10
hog: 10
kitten: 22
kitten: 22
kittens: 11
mouse: 109
opossum: 66
opossum: 66
owl: 19
owl: 19
parakeet: 14
parakeet: 14
pig: 12
pig: 12
pigeon: 242
pigeon: 242
pigeons: 6
pigs: 5
pond: 18
pond: 18
porch: 80
porch: 80
possum: 260
possum: 260
puppies: 8
puppies: 8
rabbit: 805
rabbit: 805
rabbits: 12
rabies: 62
raccoon: 838
raccoon: 838
raccoons: 34

In [33]:
merged_animal_mapping
# convert keys of merged_animal_mapping to lowercase
merged_animal_mapping = {key.lower(): value for key, value in merged_animal_mapping.items()}
# sort merged_animal_mapping by key
merged_animal_mapping = dict(sorted(merged_animal_mapping.items(), key=lambda item: item[0]))
merged_animal_mapping

{'bat, bats': 438,
 'beaver': 10,
 'bird, birds': 1983,
 'bull': 19,
 'bunnies, bunny': 82,
 'cat, cats': 1996,
 'chicken, chickens': 23,
 'chipmunk': 9,
 'coyote, coyotes': 91,
 'crow': 19,
 'deer': 45,
 'dist': 140,
 'dog': 21,
 'dog, dogs': 4317,
 'dove': 20,
 'duck, ducks': 71,
 'dumpster': 10,
 'falcon': 5,
 'ferret': 8,
 'fish': 4,
 'fox': 13,
 'geese': 8,
 'gillis': 57,
 'goose': 222,
 'groundhog': 13,
 'gull': 95,
 'gutter, gutters': 18,
 'hawk': 146,
 'hog': 10,
 'kitten, kittens': 33,
 'mouse': 109,
 'munroe': 210,
 'opossum': 66,
 'owl': 19,
 'parakeet': 14,
 'pig': 12,
 'pigeon, pigeons': 248,
 'pigs': 5,
 'pond': 18,
 'porch': 80,
 'possum': 260,
 'puppies': 8,
 'rabbit': 7,
 'rabbit, rabbits': 817,
 'rabies': 62,
 'raccoon': 11,
 'raccoon, raccoons': 872,
 'racoon': 49,
 'rooster': 13,
 'rosindale': 8,
 'scunk': 9,
 'seagul, seagull': 344,
 'seagulls': 5,
 'skunk, skunks': 566,
 'snake, snakes': 42,
 'sparrow': 8,
 'squirel, squirell': 45,
 'squirrel, squirrell': 1606,
 '

In [59]:
merged_animal_mapping = {
 'bat, bats': 438,
 'beaver': 10,
 'bird, birds': 1983,
 'bull': 19,
 'bunnies, bunny': 82,
 'cat, cats': 1996,
 'chicken, chickens': 23,
 'chipmunk': 9,
 'coyote, coyotes': 91,
 'crow': 19,
 'deer': 45,
 'dog, dog, dogs': 4338,
 'dove': 20,
 'duck, ducks': 71,
 'dumpster': 10,
 'falcon': 5,
 'ferret': 8,
 'fish': 4,
 'fox': 13,
 'geese': 8,
 'goose': 222,
 'groundhog': 13,
 'gull': 95,
 'hawk': 146,
 'hog': 10,
 'kitten, kittens': 33,
 'mouse': 109,
 'opossum': 66,
 'owl': 19,
 'parakeet': 14,
 'pig': 17,
 'pigeon, pigeons': 248,
 'possum': 260,
 'puppies': 8,
 'rabbit, rabbit, rabbits': 886,
 'raccoon, raccoon, raccoons, racoon': 932,
 'rooster': 13,
 'seagul, seagull, seagulls': 349,
 'skunk, skunks, scunk': 575,
 'snake, snakes': 42,
 'sparrow': 8,
 'squirel, squirell, squirrel, squirrell, squirrels': 1681,
 'swan': 23,
 'turkey, turkey, turkeys': 174,
 'turtle': 23}

In [57]:
# create a new dict and for every key in merged_animal_mapping, add the value to the new dict and ask the user to add the new key to the new dict
animal_to_animals = {}
for key, value in merged_animal_mapping.items():
    print(f'{key}: {value}')
    new_key = input('Enter a new key: ')
    animal_to_animals[new_key] = key

bat, bats: 438
beaver: 10
bird, birds: 1983
bull: 19
bunnies, bunny: 82
cat, cats: 1996
chicken, chickens: 23
chipmunk: 9
coyote, coyotes: 91
crow: 19
deer: 45
dog, dog, dogs: 4338
dove: 20
duck, ducks: 71
dumpster: 10
falcon: 5
ferret: 8
fish: 4
fox: 13
geese: 8
goose: 222
groundhog: 13
gull: 95
hawk: 146
hog: 10
kitten, kittens: 33
mouse: 109
opossum: 66
owl: 19
parakeet: 14
pig: 12
pigeon, pigeons: 248
pigs: 5
porch: 80
possum: 260
puppies: 8
rabbit, rabbit, rabbits, rabies: 886
raccoon, raccoon, raccoons, racoon: 932
rooster: 13
seagul, seagull, seagulls: 349
skunk, skunks, scunk: 575
snake, snakes: 42
sparrow: 8
squirel, squirell, squirrel, squirrell, squirrels: 1681
swan: 23
turkey, turkey, turkeys: 174
turtle: 23


In [3]:
animal_to_animals = {'Bats': 'bat, bats',
 'Beavers': 'beaver',
 'Birds': 'bird, birds',
 'Bulls': 'bull',
 'Bunnies': 'bunnies, bunny',
 'Cats': 'cat, cats',
 'Chicken': 'chicken, chickens',
 'Chipmunk': 'chipmunk',
 'Coyotes': 'coyote, coyotes',
 'Crows': 'crow',
 'Deers': 'deer',
 'Dogs': 'dog, dog, dogs',
 'Doves': 'dove',
 'Ducks': 'duck, ducks',
 'Falcons': 'falcon',
 'Ferrets': 'ferret',
 'Fish': 'fish',
 'Fox': 'fox',
 'Geese': 'geese',
 'Goose': 'goose',
 'Groundhog': 'groundhog',
 'Gulls': 'gull',
 'Hawks': 'hawk',
 'Hogs': 'hog',
 'Kitten': 'kitten, kittens',
 'Mouse': 'mouse',
 'Opossum': 'opossum',
 'Owl': 'owl',
 'Parakeet': 'parakeet',
 'Pigs': 'pig',
 'Pigeons': 'pigeon, pigeons',
 'Pig': 'pigs',
 'Possum': 'possum',
 'Puppies': 'puppies',
 'Rabbits': 'rabbit, rabbit, rabbits',
 'Raccoons': 'raccoon, raccoon, raccoons, racoon',
 'Roosters': 'rooster',
 'Seagulls': 'seagul, seagull, seagulls',
 'Skunks': 'skunk, skunks, scunk',
 'Snakes': 'snake, snakes',
 'Sparrows': 'sparrow',
 'Squirells': 'squirel, squirell, squirrel, squirrell, squirrels',
 'Swans': 'swan',
 'Turkeys': 'turkey, turkey, turkeys',
 'Turtles': 'turtle'}

animal_to_animals_df = pd.DataFrame(animal_to_animals.items(), columns=['animal', 'animals'])
animal_to_animals_df.to_csv('Datasets/animal_to_animals.csv', index=False)

In [5]:
filtered_animal_counts = {'Bats': 438,
 'Beavers': 10,
 'Birds': 1983,
 'Bulls': 19,
 'Bunnies': 82,
 'Cats': 1996,
 'Chicken': 23,
 'Chipmunk': 9,
 'Coyotes': 91,
 'Crows': 19,
 'Deers': 45,
 'Dogs': 4338,
 'Doves': 20,
 'Ducks': 71,
 'Falcons': 5,
 'Ferrets': 8,
 'Fish': 4,
 'Fox': 13,
 'Geese': 8,
 'Goose': 222,
 'Groundhog': 13,
 'Gulls': 95,
 'Hawks': 146,
 'Hogs': 10,
 'Kitten': 33,
 'Mouse': 109,
 'Opossum': 66,
 'Owl': 19,
 'Parakeet': 14,
 'Pigs': 12,
 'Pigeons': 248,
 'Pig': 17,
 'Possum': 260,
 'Puppies': 8,
 'Rabbits': 886,
 'Raccoons': 932,
 'Roosters': 13,
 'Seagulls': 349,
 'Skunks': 575,
 'Snakes': 42,
 'Sparrows': 8,
 'Squirells': 1681,
 'Swans': 23,
 'Turkeys': 174,
 'Turtles': 23}

filtered_animal_counts_df = pd.DataFrame(filtered_animal_counts.items(), columns=['animal', 'count'])
filtered_animal_counts_df.to_csv('Datasets/filtered_animal_counts.csv', index=False)

In [4]:
# create a new columns animal in the df and set it to nan
df['animal'] = np.nan
for i, request in enumerate(tqdm(df['closure_reason'])):
    # remove all punctuation from the request
    request_p = request.translate(str.maketrans('', '', string.punctuation))
    request_p = request_p.lower()
    # split the request into words
    words = request_p.split()
    # for every word in words, if the word is in animal_to_animals, set the animal column to the key
    for word in words:
        for animal_name, animals in animal_to_animals.items():
            for animal in animals.split(', '):
                if word == animal:
                    df.loc[df['closure_reason'] == request, 'animal'] = animal_name
                    # found a match, go to the next request
                    break  # exits the innermost loop
            else:
                continue  # continue to the next iteration of the middle loop
            break  # exits the middle loop
        else:
            continue  # continue to the next iteration of the outer loop
        break  # exits the outer loop

  0%|          | 0/83281 [00:00<?, ?it/s]/var/folders/qf/yh559xpx4dv12z0mkyw5cvr80000gn/T/ipykernel_3964/340477869.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cats' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['closure_reason'] == request, 'animal'] = animal_name
100%|██████████| 83281/83281 [01:49<00:00, 760.46it/s] 


In [5]:
# write the df to a csv file
df.to_csv('Datasets/animal_cases.csv', index=False)

In [7]:
# from GPT
previous_filtered_animals = {'Dog': 4317,
'Cat': 1996,
'Rat': 2857,
'Bird': 1983,
'Squirrel': 1631,
'Raccoon': 872,
'Rabbit': 817,
'Rodent': 655,
'Skunk': 558,
'Bat': 438,
'Possum': 260,
'Pigeon': 242,
'Goose': 222,
'Turkey': 169,
'Hawk': 146,
'Duck': 71,
'Deer': 45,
'Snake': 38,
'Turtle': 23,
'Swan': 23,
'Coyote': 91,
'Bunny': 78,
'Groundhog': 13,
'Beaver': 10,
'Chipmunk': 9,
'Chicken': 23,
'Dove': 20,
'Crow': 19,
'Owl': 19,
'Parakeet': 14,
'Robin': 12,
'Pig': 12,
'Sparrow': 8,
'Ferret': 8}

GPT_filtered_animals_df = pd.DataFrame(previous_filtered_animals.items(), columns=['animal', 'count'])
GPT_filtered_animals_df.to_csv('Datasets/GPT_filtered_animals.csv', index=False)